In [84]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from matplotlib import style
import difflib as dl


In [85]:
import matplotlib.pyplot as plt
from matplotlib import colors

def background_gradient(s, m, M, cmap='PuBu', low=0, high=0):
    rng = M - m
    norm = colors.Normalize(m - (rng * low),
                            M + (rng * high))
    normed = norm(s.values)
    c = [colors.rgb2hex(x) for x in plt.cm.get_cmap(cmap)(normed)]
    return ['background-color: %s' % color for color in c]

In [86]:
methods = pd.read_csv("methodData.csv")
modules = pd.read_csv("moduleData.csv")
teams = pd.read_csv("teamData.csv")
indAggResponses = pd.read_csv("indAggResponses.csv")
teamAggResponses = pd.read_csv("teamAggResponses.csv")
methodsChosen292 = pd.read_csv("methodsChosen292.csv")

In [87]:
# parse list representation of lists in teams to actual list
import ast
methodsChosen292['Research Methods'] = methodsChosen292['Research Methods'].apply(ast.literal_eval)
methodsChosen292['Analysis Methods'] = methodsChosen292['Analysis Methods'].apply(ast.literal_eval)
methodsChosen292['Ideate Methods'] = methodsChosen292['Ideate Methods'].apply(ast.literal_eval)
methodsChosen292['Members'] = methodsChosen292['Members'].apply(ast.literal_eval)

In [88]:
# Method Definitions for Creating Joint Distribution Tables of Method CoSelection and Exclusive Selection

def coselectJoint(modN, datatype = "prob"):
    if datatype == "prob":
        methodJoint = pd.DataFrame(columns = ["Method 1", "Method 2", "P(Method 2 Chosen | Method 1 chosen)"])
    elif datatype == "freq":
        methodJoint = pd.DataFrame(columns = ["Method 1", "Method 2", "Num Teams that Chose Method 1 and 2"])
    i = 0

    module = modules[modules["Module #"] == modN]["Module"].values[0] + " Methods"
    modMethods = methods[methods["Module #"] == modN]["Method"]
    summTeamChosen = [teamAggResponses[(teamAggResponses["Module"] == modN) 
                      & (teamAggResponses["Team"] == team)]["Chosen Method"].tolist() for team in teams["Team"]]
    

    for method1 in modMethods:
        chosen1_292 = [methList for methList in methodsChosen292[module] 
                   if len(dl.get_close_matches(method1, methList)) > 0]
        chosen1_summ = [methList for methList in summTeamChosen
                   if len(dl.get_close_matches(method1, methList)) > 0]

        # Nested lists of methods chosen by teams who chosen method1
        chosen1 = chosen1_292 + chosen1_summ

        for method2 in [method for method in modMethods]:# if method != method1]:
            # Nested lists of methods chosen by teams who chosen method1 and method2
            chosen12 = [methList for methList in chosen1 if len(dl.get_close_matches(method2, methList)) > 0]

            num_chosen1 = len(chosen1)
            num_chosen12 = len(chosen12)

            if num_chosen1:
                condProb = num_chosen12 / num_chosen1
            else:
                condProb = 0
            
            if datatype == "prob":
                methodJoint.loc[i] = [method1, method2, condProb]
            elif datatype == "freq":
                methodJoint.loc[i] = [method1, method2, float(num_chosen12)]
            i += 1
    return methodJoint

"""def compJoint(modN, datatype = "prob"):
    # datatype [str] = "prob" for joint probability distribution, or "freq" for frequency table
    
    if datatype == "prob":
        methodJoint = pd.DataFrame(columns = ["Not Method 1", "Method 2", "P(Method 2 Chosen | Method 1 not chosen)"])
    elif datatype == "freq":
        methodJoint = pd.DataFrame(columns = ["Not Method 1", "Method 2", "Num Teams that Chose Method 2 and Not Method 1"])
    i = 0

    module = modules[modules["Module #"] == modN]["Module"].values[0] + " Methods"
    modMethods = methods[methods["Module #"] == modN]["Method"]
    summTeamChosen = [teamAggResponses[(teamAggResponses["Module"] == modN) 
                      & (teamAggResponses["Team"] == team)]["Chosen Method"].tolist() for team in teams["Team"]]
    

    for method1 in modMethods:
        notchosen1_292 = [methList for methList in methodsChosen292[module] 
                   if len(dl.get_close_matches(method1, methList)) == 0]
        notchosen1_summ = [methList for methList in summTeamChosen
                   if len(dl.get_close_matches(method1, methList)) == 0]

        # Nested lists of methods chosen by teams who chosen method1
        notchosen1 = notchosen1_292 + notchosen1_summ

        for method2 in [method for method in modMethods]:
            # Nested lists of methods chosen by teams who chosen method1 and method2
            chosen2not1 = [methList for methList in notchosen1 if len(dl.get_close_matches(method2, methList)) > 0]

            num_chosen1 = len(notchosen1)
            num_chosen2not1 = len(chosen2not1)

            if num_chosen1:
                condProb = num_chosen2not1 / num_chosen1
            else:
                condProb = 0
                
            if datatype == "prob":
                methodJoint.loc[i] = [method1, method2, condProb]
            elif datatype == "freq":
                methodJoint.loc[i] = [method1, method2, float(num_chosen2not1)]
            i += 1
    return methodJoint
"""

def compJoint(modN, datatype = "prob"):
    # datatype [str] = "prob" for joint probability distribution, or "freq" for frequency table
    
    if datatype == "prob":
        methodJoint = pd.DataFrame(columns = ["Method 1", "Not Method 2", "P(Method 2 not Chosen | Method 1 chosen)"])
    elif datatype == "freq":
        methodJoint = pd.DataFrame(columns = ["Method 1", "Not Method 2", "Num Teams that Chose Method 1 and Not Method 2"])
    i = 0

    module = modules[modules["Module #"] == modN]["Module"].values[0] + " Methods"
    modMethods = methods[methods["Module #"] == modN]["Method"]
    summTeamChosen = [teamAggResponses[(teamAggResponses["Module"] == modN) 
                      & (teamAggResponses["Team"] == team)]["Chosen Method"].tolist() for team in teams["Team"]]
    

    for method1 in modMethods:
        chosen1_292 = [methList for methList in methodsChosen292[module] 
                   if len(dl.get_close_matches(method1, methList)) > 0]
        chosen1_summ = [methList for methList in summTeamChosen
                   if len(dl.get_close_matches(method1, methList)) > 0]

        # Nested lists of methods chosen by teams who chosen method1
        chosen1 = chosen1_292 + chosen1_summ

        for method2 in [method for method in modMethods]:
            # Nested lists of methods chosen by teams who chosen method1 and method2
            chosen1not2 = [methList for methList in chosen1 if len(dl.get_close_matches(method2, methList)) == 0]

            num_chosen1 = len(chosen1)
            num_chosen1not2 = len(chosen1not2)

            if num_chosen1:
                condProb = num_chosen1not2 / num_chosen1
            else:
                condProb = 0
                
            if datatype == "prob":
                methodJoint.loc[i] = [method1, method2, condProb]
            elif datatype == "freq":
                methodJoint.loc[i] = [method1, method2, float(num_chosen1not2)]
            i += 1
    return methodJoint

#coselectJoint(1).pivot("Method 1", "Method 2")

# Research

## Module 1: Team Co-Selection

### P(Method 2 was chosen | Method 1 was chosen)

In [89]:
coselectJoint(1).pivot("Method 1", "Method 2").style.apply(background_gradient,
               cmap='YlGn',
               m=coselectJoint(1)["P(Method 2 Chosen | Method 1 chosen)"].min(),
               M=coselectJoint(1)["P(Method 2 Chosen | Method 1 chosen)"].max(),
               low=0,
               high=0.2)

### Frequencies of Teams that Chose Method 1 and Method 2

In [90]:
methodJointFreq1 = coselectJoint(1, "freq")
methodJointFreq1.pivot("Method 1", "Method 2").style.apply(background_gradient,
               cmap='BuPu',
               m=methodJointFreq1["Num Teams that Chose Method 1 and 2"].min(),
               M=methodJointFreq1["Num Teams that Chose Method 1 and 2"].max(),
               low=.2,
               high= 1)

In [91]:
m=methodJointFreq1["Num Teams that Chose Method 1 and 2"].max()
m

23.0

## Module 1: Team Exclusive Selection

### P(Method 2 was chosen | Method 1 wasn't chosen)

In [92]:
methodJointComp = compJoint(1)
methodJointComp.pivot("Method 1", "Not Method 2").style.apply(background_gradient,
               cmap='PuBu',
               m=methodJointComp["P(Method 2 not Chosen | Method 1 chosen)"].min(),
               M=methodJointComp["P(Method 2 not Chosen | Method 1 chosen)"].max(),
               low=0,
               high=0.2)

### Frequencies of Teams that Chose Method 2 and Not Method 1

In [93]:
methodJointCompFreq = compJoint(1, "freq")
methodJointCompFreq.pivot("Method 1", "Not Method 2").style.apply(background_gradient,
               cmap='PuBu',
               m=methodJointCompFreq["Num Teams that Chose Method 1 and Not Method 2"].min(),
               M=methodJointCompFreq["Num Teams that Chose Method 1 and Not Method 2"].max(),
               low=0.2,
               high=.3)

# Analysis

## Module 2: Co-Selection

### P(Method 2 Chosen | Method 1 Chosen)

In [94]:
methodJoint2 = coselectJoint(2)
methodJoint2.pivot("Method 1", "Method 2").style.apply(background_gradient,
               cmap='YlGn',
               m=methodJoint2["P(Method 2 Chosen | Method 1 chosen)"].min(),
               M=methodJoint2["P(Method 2 Chosen | Method 1 chosen)"].max(),
               low=0,
               high=0.2)

### Frequencies of Teams that Chose Method 1 and 2

In [95]:
methodJoint2Freq = coselectJoint(2, "freq")
methodJoint2Freq.pivot("Method 1", "Method 2").style.apply(background_gradient,
               cmap='BuPu',
               m=methodJoint2Freq["Num Teams that Chose Method 1 and 2"].min(),
               M=methodJoint2Freq["Num Teams that Chose Method 1 and 2"].max(),
               low=.5,
               high= .8)

## Module 2: Team Exclusive Selection

### P(Method 2 Not Chosen | Method 1 Chosen)

In [96]:
methodJointComp2 = compJoint(2)
methodJointComp2.pivot("Method 1", "Not Method 2").style.apply(background_gradient,
               cmap='PuBu',
               m=methodJointComp2["P(Method 2 not Chosen | Method 1 chosen)"].min(),
               M=methodJointComp2["P(Method 2 not Chosen | Method 1 chosen)"].max(),
               low=0,
               high=0.2)

### Frequencies of Teams that Chose Method 2 and Not Method 1

In [97]:
methodJointCompFreq2 = compJoint(2, "freq")
methodJointCompFreq2.pivot("Method 1", "Not Method 2").style.apply(background_gradient,
               cmap='PuBu',
               m=methodJointCompFreq2["Num Teams that Chose Method 1 and Not Method 2"].min(),
               M=methodJointCompFreq2["Num Teams that Chose Method 1 and Not Method 2"].max(),
               low=0.2,
               high=.3)

# Ideation

## Module 3: Team Co-Selection

### P(Chose Method 2 | Chose Method 1)

In [100]:
methodJoint3 = coselectJoint(3)
methodJoint3.pivot("Method 1", "Method 2").style.apply(background_gradient,
               cmap='YlGn',
               m=methodJoint3["P(Method 2 Chosen | Method 1 chosen)"].min(),
               M=methodJoint3["P(Method 2 Chosen | Method 1 chosen)"].max(),
               low=0,
               high=0.2)

In [ ]:
### Frequencies of Teams that Chose Method 1 and 2

In [101]:
methodJoint3Freq = coselectJoint(3, "freq")
methodJoint3Freq.pivot("Method 1", "Method 2").style.apply(background_gradient,
               cmap='BuPu',
               m=methodJoint3Freq["Num Teams that Chose Method 1 and 2"].min(),
               M=methodJoint3Freq["Num Teams that Chose Method 1 and 2"].max(),
               low=.5,
               high= .8)

In [ ]:
## Module 3: Team Exclusive Selection

In [104]:
### P(Method 2 Not Chosen | Method 1 Chosen)

In [102]:
methodJointComp3 = compJoint(3)
methodJointComp3.pivot("Method 1", "Not Method 2").style.apply(background_gradient,
               cmap='PuBu',
               m=methodJointComp3["P(Method 2 not Chosen | Method 1 chosen)"].min(),
               M=methodJointComp3["P(Method 2 not Chosen | Method 1 chosen)"].max(),
               low=0,
               high=0.2)

In [ ]:
### Frequencies of Teams that Chose Method 1 and Not 2

In [103]:
methodJointCompFreq3 = compJoint(3, "freq")
methodJointCompFreq3.pivot("Method 1", "Not Method 2").style.apply(background_gradient,
               cmap='PuBu',
               m=methodJointCompFreq3["Num Teams that Chose Method 1 and Not Method 2"].min(),
               M=methodJointCompFreq3["Num Teams that Chose Method 1 and Not Method 2"].max(),
               low=0.2,
               high=.3)